In [1]:
%pip install --upgrade --quiet pip python-dotenv openai openai-agents

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import asyncio
from typing import Any
from pydantic import BaseModel
from agents import Agent, Runner, trace, WebSearchTool, FunctionTool, function_tool, RunContextWrapper, ModelSettings
from IPython.display import Markdown

In [3]:
load_dotenv(override=True)

True

In [4]:
# making your own tools the easy way

# define a pydantic model for the function arguments
class User(BaseModel):
    name: str
    age: int


@function_tool
# docstrings and signatures will be used to automatically generate a function tool schema, so include those and make them count
def make_user_dance(user: User) -> str:
    """
    Agent can make a user dance
    
    Args:
        user: User to make dance
    Returns:
        Success message
    """
    return f"User {user.name} is {user.age} years old and is now dancing"


In [5]:
# making your own tools the hard way

def do_some_simple_tool_work(data: str) -> str:
    return "done"

class FunctionArgs(BaseModel):
    username: str
    age: int

async def run_simple_tool_work(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed_args = FunctionArgs.model_validate_json(args)
    return do_some_simple_tool_work(data=f"{parsed_args.username} is {parsed_args.age} years old")

do_work_tool = FunctionTool(
    name = "process_user",
    description = "Process a user",
    params_json_schema = FunctionArgs.model_json_schema(),
    on_invoke_tool = run_simple_tool_work
)

In [6]:
simple_agent = Agent(
    name="simple_agent",
    model="gpt-4o-mini",
    instructions="""
    You are a helpful assistant that can answer questions and help with tasks.
    """,
    tools=[
        WebSearchTool(),
        do_work_tool,
        make_user_dance
    ]
)

In [7]:
run_name = "Run name example"

with trace(run_name):
    result = await Runner.run(simple_agent, "What is the current stock price of Apple? After you do that, process the user 'John' who is 30 years old. If the user is over 20, make them dance. Whats the stock price of Apple?")
    print(result.final_output)
    

The current stock price of Apple (AAPL) is $268.47. 

I've also processed the user 'John' who is 30 years old, and he is now dancing! 🎉 If you need anything else, feel free to ask!
